# result

> Conc result class.

In [ ]:
#| default_exp result

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from __future__ import annotations
import polars as pl
from fastcore.basics import patch
from great_tables import GT

In [ ]:
#| export
class Result:
	def __init__(self,
			  type: str, # report type
			  df: pl.DataFrame, # Polars dataframe with the results
			  title: str, # title of the report
			  description: str, # description
			  summary_data: dict, # summary data (ignored)
			  formatted_data: list[str] # list of formatted data about the table
			  ): 

		""" Class for results from Conc reports """
		self.type = type
		self.df = df
		self.title = title
		self.description = description
		self.summary_data = summary_data
		self.formatted_data = formatted_data


In [ ]:
#| export
@patch
def display(self: Result
			   ):
	""" Print analysis result output from conc in a nice table format using the great_tables library """
	
	columns_with_integers = []
	columns_with_decimals = []
	if self.df.select(pl.len()).item() > 0:

		self.df.columns = [col.replace('_', ' ').title() for col in self.df.columns]

		for col in self.df.columns:
			if self.df[col].dtype in [pl.Float64, pl.Float32]:
				columns_with_decimals.append(col)
			elif col != 'Token Id' and self.df[col].dtype in [pl.Int64, pl.Int32, pl.Int16, pl.Int8, pl.UInt64, pl.UInt32, pl.UInt16, pl.UInt8]:
				columns_with_integers.append(col)

	gt = GT(self.df).tab_options(table_margin_left = 0)
	if self.title != '' or self.description != '':
		gt = gt.tab_header(self.title, self.description)

	if len(columns_with_decimals) > 0:
		gt = gt.fmt_number(columns_with_decimals, decimals=2)

	if len(columns_with_integers) > 0:
		gt = gt.fmt_integer(columns_with_integers, use_seps=True, sep_mark=',')

	for value in self.formatted_data:
		gt = gt.tab_source_note(value)

	if self.type == 'concordance':
		if 'Left' in self.df.columns:
			gt = gt.cols_align(align='right', columns=['Left'])
		if 'Keyword' in self.df.columns:
			gt = gt.cols_align(align='center', columns=['Keyword'])
	gt.show()



In [ ]:
#| export
@patch
def to_frame(self: Result):
	""" Return result output from conc as a dataframe """
	return self.df

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()